In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("finance_transactions.csv")
df.head()

,transaction_id,customer_id,customer_name,transaction_date,amount,category,status,payment_method,region,merchant_name
0,1,116,Hannah Taylor,27-08-24,61.37,Income,Completed,Credit Card,West,Merchant_14
1,2,120,George King,05-08-24,546.56,Expense,Completed,Bank Transfer,West,Merchant_8
2,3,112,Jessica Lee,18-01-24,908.56,Expense,Completed,PayPal,West,Merchant_39
3,4,113,George King,30-05-24,976.36,Expense,Pending,PayPal,South,Merchant_7
4,5,105,Fiona Green,12-03-24,261.62,Expense,Completed,Debit Card,East,Merchant_30


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    500 non-null    int64  
 1   customer_id       500 non-null    int64  
 2   customer_name     500 non-null    object 
 3   transaction_date  500 non-null    object 
 4   amount            500 non-null    float64
 5   category          500 non-null    object 
 6   status            500 non-null    object 
 7   payment_method    500 non-null    object 
 8   region            500 non-null    object 
 9   merchant_name     500 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 39.2+ KB


In [4]:
df["transaction_date"] = df["transaction_date"].astype("datetime64[ms]")

In [5]:
# find the total amount of transactions and the average transaction amount for each payment method, but only for transactions with a
# status of 'Completed.' Order the results by the total amount in descending order

# Filter the dataset for transactions with 'Completed' status
completed_transactions = df[df['status'] == 'Completed']

# Group by payment method and calculate total amount and average transaction amount
result = completed_transactions.groupby('payment_method').agg(
    total_amount=('amount', 'sum'),     # Sum of the transaction amounts
    avg_amount=('amount', 'mean')       # Average of the transaction amounts
)

# Sort the result by total amount in descending order
result_sorted = result.sort_values(by='total_amount', ascending=False)

print(result_sorted)


                total_amount  avg_amount
payment_method                          
Bank Transfer       31053.94  544.805965
Credit Card         24849.08  496.981600
PayPal              23933.08  520.284348
Debit Card          22002.84  458.392500
Cash                19858.55  472.822619


In [6]:
# find the top 5 customers who have spent the highest total amount.
df.groupby("customer_name")["amount"].sum().sort_values(ascending=False).head(5)

customer_name
Hannah Taylor    34189.00
Jessica Lee      29320.15
Fiona Green      28851.42
George King      28713.34
Ian Martin       26404.12
Name: amount, dtype: float64

In [7]:
# find the total amount spent per category where the transactions were made using either 'Credit Card' or 'Debit Card' as the payment method.
# Only include categories where the total amount spent is greater than $1000. Order the results by total_amount in descending order
filter_data = df[df["payment_method"].isin(["Debit Card","Credit Card"])]
category_Total = filter_data.groupby("category")["amount"].sum()
category_Total_filter = category_Total[category_Total > 1000]
sort_value = category_Total_filter.sort_values(ascending=False)
print(sort_value)

category
Expense    53290.08
Income     51382.64
Name: amount, dtype: float64


In [8]:
# find the average transaction amount for each region where the transaction status is 'Pending' or 'Completed'. Show the region and the
# average transaction amount, and order the results by region in ascending order.
filter_status = df[
    df["status"].isin(["Pending","Completed"])
]
filter_Region = filter_status.groupby("region")["amount"].agg(
    Total_Amount = "sum", Avg_Amount = "mean")
sort_Region = filter_Region.sort_values(by = "Total_Amount",ascending=False)
print(sort_Region)


        Total_Amount  Avg_Amount
region                          
East        68291.22  521.307023
South       67859.58  521.996769
West        64674.40  497.495385
North       57949.83  531.649817


In [9]:
# find the total number of transactions and the total amount spent for each merchant in the 'Expense' category
# Show the merchant's name, the total number of transactions, and the total amount spent
filter_Category = df[
    df["category"]=="Expense"
    ]
group_category = filter_Category.groupby("merchant_name").agg(
    Total_count = ("transaction_id","count"),Total_Amount = ("amount","sum")
    )
sort_Result = group_category.sort_values(by = "Total_count", ascending=False).head(10)
print(sort_Result)

               Total_count  Total_Amount
merchant_name                           
Merchant_32             12       7175.72
Merchant_15              8       3718.64
Merchant_4               8       3966.65
Merchant_37              8       3325.55
Merchant_30              8       4657.54
Merchant_16              8       3708.10
Merchant_10              7       2492.22
Merchant_31              7       3132.52
Merchant_39              7       5136.78
Merchant_38              7       4854.18


In [10]:
# find the average transaction amount for each region where the payment_method is 'Cash' and the status is 'Completed', but only for
# transactions that occurred in the first quarter of 2024 (January, February, and March). Show the region and the average transaction amount.
filter_data = df[
    (df["payment_method"]== "Cash") & 
    (df["status"]== "Completed") & 
    (df["transaction_date"].between('2024-01-01','2024-03-31'))
    ]
Result_data = filter_data.groupby("region")["amount"].mean()
print(Result_data)

region
East     568.73
North     53.97
South    434.54
Name: amount, dtype: float64


In [11]:
# find the total amount spent in the 'Income' category for each customer, but only for transactions where the payment method is either 'Bank
# Transfer' or 'PayPal'. Show the customer_name, customer_id, and the total amount spent in the 'Income' category.
filterd_data  = df[
    (df["category"] == "Income") &
    (df["payment_method"].isin(["Bank Transfer","PayPal"]))]
Result_Groupby = filterd_data.groupby(
    ["customer_name","customer_id"])["amount"].sum().reset_index()
print(Result_Groupby)

    customer_name  customer_id   amount
0   Alice Johnson          105   702.91
1   Alice Johnson          107   507.03
2   Alice Johnson          108   488.00
3   Alice Johnson          109   983.23
4   Alice Johnson          111   674.23
..            ...          ...      ...
78    Jessica Lee          114   957.34
79    Jessica Lee          115   166.72
80    Jessica Lee          117  1374.08
81    Jessica Lee          118   925.91
82    Jessica Lee          120   456.07

[83 rows x 3 columns]


In [12]:
# find the merchant(s) who have the highest total transaction amount in the 'Expense' category.
# Show the merchant_name and the total amount spent for that merchant. 
filterd_merchant = df[
    df["category"]== "Expense"
    ]
result_Merchant = filterd_merchant.groupby(
    "merchant_name")["amount"].sum().reset_index().sort_values(
        by = "amount", ascending=False).head(1)
print(result_Merchant)

   merchant_name   amount
25   Merchant_32  7175.72


In [13]:
# find the total number of transactions and the total amount for each payment method in the 'Expense' category. Only include the results for payment
# methods that have more than 5 transactions. Show the payment_method, the total number of transactions, and the total amount spent for each payment method
filterd_merchant_category = df[
    df["category"]== "Expense"
    ]
result_filterd_merchant_category = filterd_merchant_category.groupby("payment_method").agg(
    Total_Count = ("transaction_id","count"), Total_amount = ("amount","sum")
)
print(result_filterd_merchant_category)

                Total_Count  Total_amount
payment_method                           
Bank Transfer            61      33728.35
Cash                     46      24195.73
Credit Card              46      26043.36
Debit Card               52      27246.72
PayPal                   40      21290.82


In [14]:
# find the average transaction amount for each customer in the 'Expense' category. Show the customer_name, customer_id, and the average transaction amount
filterd_merchantcategory = df[
    df["category"]== "Expense"
    ]
result_filterd_merchantcategory = filterd_merchantcategory.groupby(
    ["customer_name","customer_id"])["amount"].mean().reset_index()
print(result_filterd_merchantcategory)

     customer_name  customer_id  amount
0    Alice Johnson          101   55.17
1    Alice Johnson          102  487.84
2    Alice Johnson          103  669.80
3    Alice Johnson          104  254.19
4    Alice Johnson          105  228.16
..             ...          ...     ...
130    Jessica Lee          116  171.60
131    Jessica Lee          117  168.63
132    Jessica Lee          118  708.49
133    Jessica Lee          119  620.85
134    Jessica Lee          120  630.39

[135 rows x 3 columns]


In [15]:
# find the top 3 merchants in terms of the average transaction amount in the 'Expense' category. Show
# the merchant_name and the average transaction amount for each of the top 3 merchants
filter_merchents = df[
    df["category"]== "Expense"
    ]
groupby_merchant = filter_merchents.groupby(
    "merchant_name")["amount"].mean().sort_values(ascending=False).head(3).reset_index()
print(groupby_merchant)

  merchant_name      amount
0   Merchant_40  807.063333
1    Merchant_9  781.614000
2   Merchant_49  742.313333


In [16]:
# find the top 3 regions based on the total amount for the 'Expense' category, ordered by
# total amount in descending order. Make sure to display the region and the total amount
filter_data_merchents = df[
    df["category"]== "Expense"
    ]
groupby_Regions = filter_data_merchents.groupby(
    ["region"])["amount"].sum().sort_values(ascending=False).reset_index().head(3)
print(groupby_Regions)

  region    amount
0   West  40080.65
1   East  33071.90
2  South  31247.90


In [17]:
# find the average amount per payment method for the 'Income' category, but only for transactions that have a status of 'Completed'.
# Make sure to group the results by payment method and order the results by average amount in descending order
filter_payment_method = df[
    (df["category"]== "Income") & 
    (df["status"]== "Completed")
    ]
result_Payment_Method = filter_payment_method.groupby(
    "payment_method")["amount"].mean().sort_values(ascending=False).reset_index()
result_Payment_Method.columns = ("payment_method", "Avg_Amount")
print(result_Payment_Method)

  payment_method  Avg_Amount
0  Bank Transfer  615.324500
1         PayPal  488.192667
2           Cash  452.364737
3    Credit Card  445.305172
4     Debit Card  417.993750


In [18]:
# find the total number of transactions and the total amount for each customer where the payment method is
# either 'Credit Card' or 'Debit Card'. Group the results by customer_name and customer_id, and order them by total amount in descending order
filter_data_payment_method = df[
    df["payment_method"].isin(["Credit Card","Debit Card"])
    ]
groupby_customer_name = filter_data_payment_method.groupby(
    ["customer_name","customer_id"]).agg(
        Total_Trasection = ("transaction_id","count"),
        Total_Amount = ("amount","sum")
).sort_values(
    by ="Total_Amount", ascending=False
    )
print(groupby_customer_name)

                           Total_Trasection  Total_Amount
customer_name customer_id                                
Jessica Lee   102                         4       2691.72
Ian Martin    104                         3       2309.12
Hannah Taylor 113                         4       2194.55
Bob Smith     105                         3       1908.48
Hannah Taylor 105                         3       1793.06
...                                     ...           ...
Charlie Brown 108                         1        139.62
Bob Smith     115                         1        138.15
Ethan Black   110                         1         95.86
Fiona Green   104                         1         90.30
Alice Johnson 119                         1         71.71

[135 rows x 2 columns]


In [19]:
# find the merchant name and the total amount for each merchant in the 'Income' category where
# the transaction status is 'Completed'. Display the merchants who have a total amount greater than 500
filter_categoty_Status = df[
    (df["category"]== "Income") & 
    (df["status"]== "Completed") & 
    (df["amount"]> 500)
    ]
Result_categoty_Status = filter_categoty_Status.groupby(
    "merchant_name")["amount"].sum().sort_values(ascending=False).reset_index()
Result_categoty_Status.columns = ("Merchant_Name","Total_Amount")
print(Result_categoty_Status)

   Merchant_Name  Total_Amount
0    Merchant_27       3167.81
1    Merchant_16       2408.38
2    Merchant_34       2333.73
3    Merchant_28       2227.31
4     Merchant_3       2203.35
5    Merchant_18       2132.43
6     Merchant_4       2056.14
7    Merchant_45       1989.14
8    Merchant_11       1585.49
9    Merchant_23       1538.66
10   Merchant_30       1393.36
11    Merchant_9       1328.10
12   Merchant_48       1142.08
13   Merchant_42       1133.73
14   Merchant_26        983.48
15    Merchant_8        983.23
16    Merchant_7        952.32
17   Merchant_38        927.22
18   Merchant_40        897.45
19    Merchant_1        883.41
20   Merchant_49        867.48
21   Merchant_43        850.38
22   Merchant_33        817.88
23   Merchant_17        750.64
24   Merchant_44        723.61
25   Merchant_47        684.33
26   Merchant_32        670.96
27   Merchant_21        646.64
28   Merchant_37        608.60
29   Merchant_41        605.77
30   Merchant_22        598.48
31   Mer

In [20]:
# find the top 5 customers who have spent the most in the 'Expense' category in year 2024, showing
# customer name, customer id, and the total amount spent. Order the results by total amount in descending order
filter_expense_category = df[
    (df["category"]=="Expense") & 
    (pd.to_datetime(df["transaction_date"]).dt.year == 2024)
    ]
result_customer = filter_expense_category.groupby(
    ["customer_name","customer_id"])["amount"].sum().sort_values(ascending=False).reset_index().head(5)
print(result_customer)


   customer_name  customer_id   amount
0    Fiona Green          103  3550.88
1  Hannah Taylor          105  3097.61
2    George King          120  3001.13
3  Hannah Taylor          120  2875.99
4  Hannah Taylor          113  2517.25


In [21]:
# find the average transaction amount for each region in the 'Income' category, but only for
# transactions where the payment method is 'Bank Transfer'. Group the results by region and payment method, and order them by region
filter_ctegory_and_payment_method = df[
    (df["category"] == "Income") & 
    (df["payment_method"]=="Bank Transfer")
    ]
groupby_payment_Method = filter_ctegory_and_payment_method.groupby(
    ["region","payment_method"])["amount"].mean().reset_index().sort_values(by = "region")
groupby_payment_Method.columns = (
    "Region","Payment_Method","Avg_Amount"
    )
print(groupby_payment_Method)

  Region Payment_Method  Avg_Amount
0   East  Bank Transfer  543.903636
1  North  Bank Transfer  687.570000
2  South  Bank Transfer  573.516154
3   West  Bank Transfer  508.428333


In [22]:
# find the total amount and the number of transactions for each month in 2024 for the 'Expense'
# category. Display the month, total amount, and transaction count, and order the results by month in ascending order

filter_expense_2024 = df[
    (df["category"] == "Expense") & 
    (df["transaction_date"].dt.year == 2024)
]
filter_expense_2024["month"] = filter_expense_2024["transaction_date"].dt.month
result_monthly = (
    filter_expense_2024.groupby("month")
    .agg(
        total_amount=("amount", "sum"),
        transaction_count=("transaction_id", "count")
    )
    .reset_index()
    .sort_values(by="month")
)

print(result_monthly)


    month  total_amount  transaction_count
0       1      12078.04                 18
1       2      12696.82                 22
2       3      11710.51                 20
3       4       5732.07                 13
4       5      12949.09                 25
5       6       7814.70                 17
6       7      11301.86                 24
7       8      13164.80                 23
8       9      12240.89                 24
9      10       8774.74                 16
10     11      13651.53                 21
11     12      10389.93                 22


C:\Users\Waseem Abbas\AppData\Local\Temp\ipykernel_7512\3759953935.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_expense_2024["month"] = filter_expense_2024["transaction_date"].dt.month


In [23]:
# find the total number of transactions, total amount, and average amount for each status in the
# 'Income' category. Group the results by status and order by total amount in descending order.
filter_by_category = df[
    df["category"]=="Income"
    ]
groupby_filter_by_category = filter_by_category.groupby("status").agg(
    Total_Transections = ("transaction_id","count"),
    Total_Amount = ("amount","sum"),
    Average_amount = ("amount","mean")
).sort_values(by = "Total_Amount",ascending=False)
print(groupby_filter_by_category)

           Total_Transections  Total_Amount  Average_amount
status                                                     
Pending                   133      67777.15      509.602632
Completed                 122      58492.90      479.450000


In [24]:
# calculate the cumulative total amount for each region in the 'Income' category
# calculate this cumulative total, and display the columns: region, transaction_date, amount,
# and the cumulative total. Order the results by region and transaction_date.

filter_income = df[
    df["category"] == "Income"
    ]
filter_income = filter_income.sort_values(
    by=["region", "transaction_date"]
    )
filter_income["cumulative_total"] = filter_income.groupby("region")["amount"].cumsum()
result = filter_income[
    ["region", "transaction_date", "amount", "cumulative_total"]
    ]
print(result)


    region transaction_date  amount  cumulative_total
125   East       2024-01-06  560.62            560.62
54    East       2024-01-16  152.67            713.29
476   East       2024-01-17  595.75           1309.04
261   East       2024-02-12  147.32           1456.36
150   East       2024-02-15  142.08           1598.44
..     ...              ...     ...               ...
51    West       2024-12-11  901.74          23236.68
189   West       2024-12-18  651.02          23887.70
262   West       2024-12-23  171.90          24059.60
457   West       2024-12-23  319.92          24379.52
340   West       2024-12-30  214.23          24593.75

[255 rows x 4 columns]


In [25]:
# find the top 3 customers who have spent the most in the 'Expense' category in 2024. For each customer, display their customer name, customer ID,
# and total amount spent. The results should be ordered by total amount spent in descending order.
filter_category_Expense = df[
    (df["category"] == "Expense") & 
    (df["transaction_date"].dt.year == 2024)
    ]
groupby_customer = filter_category_Expense.groupby(
    ["customer_name","customer_id"])["amount"].sum().sort_values(ascending=False).reset_index().head(3)
groupby_customer.columns = ("customer_name","customer_id","Total_Amount")
print(groupby_customer)

   customer_name  customer_id  Total_Amount
0    Fiona Green          103       3550.88
1  Hannah Taylor          105       3097.61
2    George King          120       3001.13


In [26]:
# find the first and last transaction amount for each merchant in the 'Income' category in 2024.
# Display the merchant name, first transaction amount, and last transaction amount, and order the results by merchant name
filter_category_income = df[
    (df["category"] == "Income") & 
    (df["transaction_date"].dt.year == 2024)
]
Sort_Order = filter_category_income.sort_values(["merchant_name","transaction_date"])

Result_filter_category_income = Sort_Order.groupby("merchant_name").agg(
    First_Trasection = ("transaction_id","first"),Last_Trasection = ("transaction_id","last") 
)
print(Result_filter_category_income)

               First_Trasection  Last_Trasection
merchant_name                                   
Merchant_1                  338              297
Merchant_10                  33              232
Merchant_11                 499               47
Merchant_12                 179              392
Merchant_13                 181              379
Merchant_14                 194              491
Merchant_15                 207              271
Merchant_16                 366                9
Merchant_17                 436              198
Merchant_18                 126              369
Merchant_19                 262              309
Merchant_2                  152               63
Merchant_20                 153              153
Merchant_21                 390              257
Merchant_22                  55               67
Merchant_23                 177               90
Merchant_24                 489              197
Merchant_25                 228               91
Merchant_26         

In [27]:
# find the average transaction amount for each payment method in the 'Expense' category, but only
# for transactions that occurred in the second quarter (April, May, June) of 2024. Display the payment method and the average transaction amount.
filter_by_category_Expense = df[
    (df["category"] == "Expense") & 
    (df["transaction_date"].dt.month).between(4,6) &
     (df["transaction_date"].dt.year == 2024)
    ]
groupby_filter_by_category_Expense = filter_by_category_Expense.groupby("payment_method")["amount"].mean().reset_index()
print(groupby_filter_by_category_Expense)

  payment_method      amount
0  Bank Transfer  435.668571
1           Cash  507.691429
2    Credit Card  482.689231
3     Debit Card  397.405714
4         PayPal  604.574286


In [28]:
# determine the total number of transactions and the total amount for each region in the 'Income'  category, but only for transactions where the status is
# either 'Completed' or 'Pending'. Display the region, total number of transactions, and total amount, sorted by the total amount in descending order
filter_by_category_Status = df[
    (df["category"] == "Income") &
    (df["status"].isin(["Completed","Pending"]))
    ]
groupby_filter_by_category_Status = filter_by_category_Status.groupby("region").agg(
    Total_Trasection = ("transaction_id","count"),
    Total_Amount = ("amount","sum")
).sort_values(by = "Total_Amount", ascending=False)
print(groupby_filter_by_category_Status)


        Total_Trasection  Total_Amount
region                                
South                 70      36611.68
East                  69      35219.32
North                 55      29845.30
West                  61      24593.75


In [29]:
# find the month with the highest total transaction amount for the 'Expense' category in 2024.
# Display the month name and the total amount.
filter_by_category_Expense = df[
    (df["category"] == "Expense") &
    (df["transaction_date"].dt.year == 2024)
    ].copy()
filter_by_category_Expense["Month"]= (df["transaction_date"].dt.month)
groupby_filter_by_category_Expense = filter_by_category_Expense.groupby("Month")["amount"].sum().sort_values(ascending=False).reset_index().head(1)
print(groupby_filter_by_category_Expense)

   Month    amount
0     11  13651.53


In [30]:
# calculate the cumulative transaction amount for the 'Income' category, sorted by transaction
# date. Display the transaction date, customer name, transaction amount, and the cumulative total amount
filter_income_category = df[df["category"]=="Income"]
sort_filter_income_category = filter_income_category.sort_values(by = "transaction_date")
sort_filter_income_category["Comulative_Total"] = sort_filter_income_category["amount"].cumsum()
sort_filter_income_category_result =sort_filter_income_category[["transaction_date","customer_name","amount","Comulative_Total"]]
print(sort_filter_income_category_result)

    transaction_date  customer_name  amount  Comulative_Total
260       2024-01-03      Bob Smith   50.15             50.15
254       2024-01-03    George King  733.66            783.81
331       2024-01-05  Charlie Brown  987.68           1771.49
351       2024-01-05      Bob Smith  594.23           2365.72
449       2024-01-06    Fiona Green  907.91           3273.63
..               ...            ...     ...               ...
121       2024-12-27  Hannah Taylor  199.80         124883.90
62        2024-12-28    Ethan Black  115.53         124999.43
269       2024-12-30    Ethan Black  451.85         125451.28
340       2024-12-30    Ethan Black  214.23         125665.51
391       2024-12-30    Jessica Lee  604.54         126270.05

[255 rows x 4 columns]


In [31]:
# find the percentage contribution of each payment method to the total transaction amount in the
# 'Expense' category. Display the payment method, total amount, and percentage contribution
filter_expense = df[df["category"] == "Expense"]
grouped_expense = filter_expense.groupby("payment_method")["amount"].sum().reset_index()
total_amount = grouped_expense["amount"].sum()
grouped_expense["Percentage_Contribution"] = (grouped_expense["amount"] / total_amount) * 100
grouped_expense.columns = ["Payment_Method", "Total_Amount", "Percentage_Contribution"]
print(grouped_expense)

  Payment_Method  Total_Amount  Percentage_Contribution
0  Bank Transfer      33728.35                25.454402
1           Cash      24195.73                18.260242
2    Credit Card      26043.36                19.654627
3     Debit Card      27246.72                20.562789
4         PayPal      21290.82                16.067939


In [32]:
# find the top 3 regions with the highest average income per transaction. Display the region and
# the average income amount, sorted in descending order
# Filter the data for 'Income' category
filter_income = df[df["category"] == "Income"]

# Group by region and calculate the average transaction amount
average_income = (
    filter_income
    .groupby("region", as_index=False)
    .agg(Average_Income=("amount", "mean"))
    .sort_values(by="Average_Income", ascending=False)
    .head(3)
)
print(average_income)


  region  Average_Income
1  North      542.641818
2  South      523.024000
0   East      510.424928


In [33]:
# determine the month with the lowest total expense. Display the month name and the total
# expense amount, sorted in ascending order by total expense
filtered_Expense_Category = df[
    (df["category"] == "Expense")].copy() 

filtered_Expense_Category["Months"] = filtered_Expense_Category["transaction_date"].dt.month
filtered_Expense_Category_groupby = filtered_Expense_Category.groupby("Months")["amount"].sum().reset_index().sort_values(by = "amount", ascending=True).head(1)
print(filtered_Expense_Category_groupby)

   Months   amount
3       4  5732.07


In [34]:
# calculate the total number of transactions, the total amount, and the average
# transaction amount for each payment method and category. Sort the results by category and then by payment method.

groupby_Result = df.groupby(["category","payment_method"]).agg(
    Total_Transaction = ("transaction_id","count"),
    Total_Amount = ("amount","sum"),
    Avg_Trnsaction_Amount = ("amount","mean")
)
sort_Result = groupby_Result.sort_values(by = ["category","payment_method"])
print(sort_Result)

                         Total_Transaction  Total_Amount  \
category payment_method                                    
Expense  Bank Transfer                  61      33728.35   
         Cash                           46      24195.73   
         Credit Card                    46      26043.36   
         Debit Card                     52      27246.72   
         PayPal                         40      21290.82   
Income   Bank Transfer                  46      26415.49   
         Cash                           42      19523.15   
         Credit Card                    58      26533.96   
         Debit Card                     51      24848.68   
         PayPal                         58      28948.77   

                         Avg_Trnsaction_Amount  
category payment_method                         
Expense  Bank Transfer              552.923770  
         Cash                       525.994130  
         Credit Card                566.160000  
         Debit Card               

In [35]:
# calculate the total income and total expense for each region. Display the region, total income,
# total expense, and the net balance (total income - total expense). Sort the results by the  net balance in descending order

# Filter income and expense data
income_data = df[df["category"] == "Income"]
expense_data = df[df["category"] == "Expense"]

# Group by region and calculate total income
income_by_region = income_data.groupby("region")["amount"].sum().reset_index()
income_by_region = income_by_region.rename(columns={"amount": "Total_Income"})  # Assign back

# Group by region and calculate total expense
expense_by_region = expense_data.groupby("region")["amount"].sum().reset_index()
expense_by_region = expense_by_region.rename(columns={"amount": "Total_Expense"})  # Assign back

# Merge the two results on 'region'
merged_data = pd.merge(income_by_region, expense_by_region, on="region", how="outer").fillna(0)

# Calculate the net balance (Total Income - Total Expense)
merged_data["Net_Balance"] = merged_data["Total_Income"] - merged_data["Total_Expense"]

# Sort by net balance in descending order
sorted_result = merged_data.sort_values(by="Net_Balance", ascending=False)

# Display the result
print(sorted_result)



  region  Total_Income  Total_Expense  Net_Balance
2  South      36611.68       31247.90      5363.78
0   East      35219.32       33071.90      2147.42
1  North      29845.30       28104.53      1740.77
3   West      24593.75       40080.65    -15486.90


In [36]:
# display the top 3 merchants in terms of total income, along with the corresponding total
# transaction count and total income amount. Sort the results by total income in descending order
df_merchant_groupby = df.groupby("merchant_name").agg(
    Total_Income = ("amount","sum"),
    Total_Transaction = ("transaction_id","count")
).sort_values(by = "Total_Income", ascending=False).head(3)
print(df_merchant_groupby)

               Total_Income  Total_Transaction
merchant_name                                 
Merchant_32         8601.25                 15
Merchant_39         8129.96                 11
Merchant_4          8029.98                 15


In [37]:
# find the percentage contribution of each region to the total income. Sort the results in descending
# order of their percentage contribution

income_data = df[df["category"] == "Income"]

income_by_region = income_data.groupby("region")["amount"].sum().reset_index()
income_by_region.rename(columns={"amount": "Total_Income"}, inplace=True)
total_income = income_by_region["Total_Income"].sum()
income_by_region["Percentage_Contribution"] = (income_by_region["Total_Income"] / total_income) * 100
sorted_income = income_by_region.sort_values(by="Percentage_Contribution", ascending=False)
print(sorted_income)


  region  Total_Income  Percentage_Contribution
2  South      36611.68                28.994746
0   East      35219.32                27.892061
1  North      29845.30                23.636088
3   West      24593.75                19.477105


In [38]:
# display the monthly trend of the total amount (both income and expense). Include the month name,
# category, and the total amount. Sort the results first by category and then by month in ascending order

df["month"] = df["transaction_date"].dt.strftime("%B")  # Extract the month name

# Define a custom order for months to ensure proper sorting
month_order = [
    "January", "February", "March", "April", "May", "June", 
    "July", "August", "September", "October", "November", "December"
]

# Group by month and category, and calculate the total amount
monthly_trend = (
    df.groupby(["month", "category"])["amount"]
    .sum()
    .reset_index()
    .rename(columns={"amount": "Total_Amount"})
)

# Add a month number column for sorting
monthly_trend["month_number"] = monthly_trend["month"].apply(lambda x: month_order.index(x) + 1)

# Sort the results by category and then by month number
sorted_trend = monthly_trend.sort_values(by=["category", "month_number"]).drop(columns="month_number")

print(sorted_trend)


        month category  Total_Amount
8     January  Expense      12078.04
6    February  Expense      12696.82
14      March  Expense      11710.51
0       April  Expense       5732.07
16        May  Expense      12949.09
12       June  Expense       7814.70
10       July  Expense      11301.86
2      August  Expense      13164.80
22  September  Expense      12240.89
20    October  Expense       8774.74
18   November  Expense      13651.53
4    December  Expense      10389.93
9     January   Income      14774.74
7    February   Income       9637.62
15      March   Income      16691.56
1       April   Income       7266.69
17        May   Income       7836.04
13       June   Income       8043.69
11       July   Income      10376.45
3      August   Income      11833.55
23  September   Income       8809.51
21    October   Income      10731.39
19   November   Income      11708.98
5    December   Income       8559.83


In [39]:
# find the average amount for each payment method in each region, but only for transactions in the
# 'Expense' category. The result should be ordered by payment method in ascending order
filterregion = df[df["category"]== "Expense"]
filterregion_groupby = filterregion.groupby(["region","payment_method"])["amount"].mean().reset_index().rename(
    columns={"amount": "Avg_Amount"}).sort_values(
        by ="payment_method", ascending=True)

print(filterregion_groupby)

   region payment_method  Avg_Amount
0    East  Bank Transfer  719.258182
5   North  Bank Transfer  422.180000
10  South  Bank Transfer  542.981429
15   West  Bank Transfer  572.994500
1    East           Cash  545.532353
6   North           Cash  600.167500
11  South           Cash  477.110000
16   West           Cash  488.335556
2    East    Credit Card  458.745000
17   West    Credit Card  646.342667
7   North    Credit Card  609.343333
12  South    Credit Card  487.830909
3    East     Debit Card  527.505625
8   North     Debit Card  492.387273
18   West     Debit Card  578.535000
13  South     Debit Card  495.996154
9   North         PayPal  545.704286
4    East         PayPal  377.596000
14  South         PayPal  610.675000
19   West         PayPal  583.706154


In [40]:
# find the total amount of 'Income' transactions for each merchant name. Include only
# merchants with a total amount greater than 1,000, and order the result by the total amount in descending order
income_data = df[df["category"] == "Income"]

# Group by merchant name and calculate the total amount
total_income_by_merchant = (
    income_data.groupby("merchant_name")["amount"]
    .sum()
    .reset_index()
    .rename(columns={"amount": "Total_Income"})
)

filtered_merchants = total_income_by_merchant[total_income_by_merchant["Total_Income"] > 1000]
sorted_merchants = filtered_merchants.sort_values(by="Total_Income", ascending=False)

print(sorted_merchants)


   merchant_name  Total_Income
19   Merchant_27       7355.84
15   Merchant_23       4949.61
33    Merchant_4       4063.33
20   Merchant_28       3922.13
27   Merchant_34       3810.27
44    Merchant_5       3790.04
29   Merchant_36       3704.51
39   Merchant_45       3597.24
48    Merchant_8       3423.95
42   Merchant_48       3340.30
41   Merchant_47       3331.28
9    Merchant_18       3297.55
38   Merchant_44       3286.61
22    Merchant_3       3227.72
2    Merchant_11       3206.29
34   Merchant_40       3177.82
36   Merchant_42       3142.30
32   Merchant_39       2993.18
13   Merchant_21       2945.17
49    Merchant_9       2939.47
47    Merchant_7       2892.25
7    Merchant_16       2744.66
16   Merchant_24       2697.08
3    Merchant_12       2626.03
30   Merchant_37       2608.50
35   Merchant_41       2518.48
18   Merchant_26       2393.15
17   Merchant_25       2315.69
1    Merchant_10       2290.14
4    Merchant_13       2212.65
43   Merchant_49       2168.02
46    Me

In [41]:
# count the number of transactions and sum the amounts for each status ('Completed' and 'Pending')
# within the 'Expense' category, and order the result by status in ascending order.
filterexpense_category = df[df["category"]== "Expense"]
groupdata = filterexpense_category.groupby("status").agg(
    Total_Transaction = ("transaction_id","count"),
    Total_Amount = ("amount","sum")
).sort_values(by = "status", ascending=True)
print(groupdata)


           Total_Transaction  Total_Amount
status                                    
Completed                121      63204.59
Pending                  124      69300.39


In [42]:
# find the total amount and average amount for each payment method in the 'Income' category. Only
# include rows where the average amount is greater than 500, and order the result by payment method in ascending order
income_ctegory_filter = df[df["category"]== "Income"]
groupby_paymentmethod = income_ctegory_filter.groupby("payment_method").agg(
    Total_Amount = ("amount","sum"),
    Avg_amount = ("amount","mean")
)
greaterthan_500 = groupby_paymentmethod[groupby_paymentmethod["Avg_amount"]> 500]
greaterthan_500 = greaterthan_500.sort_index()
print(greaterthan_500)

                Total_Amount  Avg_amount
payment_method                          
Bank Transfer       26415.49  574.249783


In [43]:
# find the top 3 regions with the highest total amount of 'Income' transactions in 2024, but only
# include those regions where the average transaction amount for that region is greater than 300. Order the results by
# total amount in descending order
filet_category_for_regions = df[
    (df["category"]== "Income") &
    (df["transaction_date"].dt.year == 2024)
    ]
group_by_regions = filet_category_for_regions.groupby("region").agg(
    Total_Amount = ("amount","sum"),
    avg_Amount = ("amount","mean")
).sort_index()
greaterthan300 = group_by_regions[group_by_regions["avg_Amount"]> 300]
order_by_Total_amount = greaterthan300.sort_values(by = "Total_Amount", ascending=False).head(3)
print(order_by_Total_amount)

        Total_Amount  avg_Amount
region                          
South       36611.68  523.024000
East        35219.32  510.424928
North       29845.30  542.641818


In [44]:
# find the difference between the highest and lowest transaction amount for each merchant in the
# 'Income' category. Return the merchant_name, the difference (as Amount_Difference), and order by the largest difference first.
income_category = df[df["category"] == "Income"]
merchant_diff = income_category.groupby("merchant_name").agg(
    Max_Amount=("amount", "max"),
    Min_Amount=("amount", "min")
)

merchant_diff["Amount_Difference"] = merchant_diff["Max_Amount"] - merchant_diff["Min_Amount"]

sorted_diff = merchant_diff.sort_values(by="Amount_Difference", ascending=False)
print(sorted_diff)


               Max_Amount  Min_Amount  Amount_Difference
merchant_name                                           
Merchant_8         983.23       50.15             933.08
Merchant_18        995.38       71.71             923.67
Merchant_40        957.34       54.38             902.96
Merchant_21        942.91       71.68             871.23
Merchant_23        932.45       69.30             863.15
Merchant_28        977.63      124.53             853.10
Merchant_6         894.90       66.30             828.60
Merchant_27        987.68      163.69             823.99
Merchant_1         883.41       75.10             808.31
Merchant_34        965.90      164.49             801.41
Merchant_44        926.48      141.14             785.34
Merchant_43        850.38       78.96             771.42
Merchant_4         907.91      142.08             765.83
Merchant_14        818.29       61.37             756.92
Merchant_16        903.28      147.66             755.62
Merchant_7         952.32      

In [45]:
# find the top 5 customers who spent the most in total in the 'Income' category, but only consider
# transactions where the payment method is 'Credit Card' or 'Debit Card'. You need to return the customer name, customer ID, and total amount spent
income_category_filter = df[
    (df["category"] == "Income") & 
    (df["payment_method"].isin(["Credit Card","Debit Card"]))
    ]
income_category_filter_groupby = income_category_filter.groupby(    
    ["customer_name","customer_id"])["amount"].sum().reset_index().sort_values(
        by = "amount",ascending=False).head(5)
print(income_category_filter_groupby)

    customer_name  customer_id   amount
74    Jessica Lee          102  1810.31
43    Fiona Green          111  1666.70
36    Ethan Black          112  1604.79
20  Charlie Brown          118  1551.89
45    Fiona Green          115  1399.91


In [46]:
# calculate the cumulative sum of amount for each transaction, ordered by transaction_date? For this,
# assume the dataset has columns: transaction_id, transaction_date, and amount

df_sorted = df.sort_values(by="transaction_date")  
df_sorted["cumulative_amount"] = df_sorted["amount"].cumsum()  

print(df_sorted[["transaction_id", "transaction_date", "amount", "cumulative_amount"]])



     transaction_id transaction_date  amount  cumulative_amount
330             331       2024-01-02  881.41             881.41
254             255       2024-01-03  733.66            1615.07
260             261       2024-01-03   50.15            1665.22
228             229       2024-01-04  231.88            1897.10
319             320       2024-01-05  967.23            2864.33
..              ...              ...     ...                ...
62               63       2024-12-28  115.53          257284.21
340             341       2024-12-30  214.23          257498.44
269             270       2024-12-30  451.85          257950.29
24               25       2024-12-30  220.20          258170.49
391             392       2024-12-30  604.54          258775.03

[500 rows x 4 columns]


In [47]:
# calculate the running total of the amount for each region, considering each region
# should have its own cumulative total starting from the first transaction?

df_sorted = df.sort_values(by=["region", "transaction_date"])  
df_sorted["running_total"] = df_sorted.groupby("region")["amount"].cumsum()

print(df_sorted[["transaction_id", "region", "transaction_date", "amount", "running_total"]])


     transaction_id region transaction_date  amount  running_total
228             229   East       2024-01-04  231.88         231.88
125             126   East       2024-01-06  560.62         792.50
426             427   East       2024-01-07  764.72        1557.22
54               55   East       2024-01-16  152.67        1709.89
476             477   East       2024-01-17  595.75        2305.64
..              ...    ...              ...     ...            ...
410             411   West       2024-12-20  342.86       63283.55
262             263   West       2024-12-23  171.90       63455.45
457             458   West       2024-12-23  319.92       63775.37
286             287   West       2024-12-26  684.80       64460.17
340             341   West       2024-12-30  214.23       64674.40

[500 rows x 5 columns]


In [48]:
# find the top 3 merchants based on the highest total amount in a specific category (e.g., 'Income') within each region
filtered_data = df[df["category"] == "Income"]
grouped_data = filtered_data.groupby(
    ["region", "merchant_name"])["amount"].sum().reset_index()
sorted_data = grouped_data.sort_values(
    by=["region", "amount"], ascending=[True, False])
top_3_merchants_per_region = sorted_data.groupby("region").head(3)
print(top_3_merchants_per_region)


    region merchant_name   amount
2     East   Merchant_11  3206.29
6     East   Merchant_16  2408.38
8     East   Merchant_18  2220.29
41   North   Merchant_23  1965.89
42   North   Merchant_24  1800.70
51   North   Merchant_36  1638.26
79   South   Merchant_27  3926.04
80   South   Merchant_28  2227.31
92   South   Merchant_44  2121.14
141   West    Merchant_8  1757.70
115   West   Merchant_23  1439.51
118   West   Merchant_27  1270.48


In [52]:
# find the total number of transactions, total income, and average transaction amount for each month in 2024, sorted by month
# Filter the data for the "Income" category
income_data = df[(df["category"] == "Income") & (df["transaction_date"].dt.year == 2024)]

# Group by month and calculate total transactions, total income, and average transaction amount
monthly_income_stats = income_data.groupby(income_data["transaction_date"].dt.month).agg(
    Total_Transactions=("transaction_id", "count"),
    Total_Income=("amount", "sum"),
    Avg_Transaction_Amount=("amount", "mean")
).reset_index()

# Rename the month column for better readability
monthly_income_stats.rename(columns={"transaction_date": "Month"}, inplace=True)

# Sort the results by month in ascending order
monthly_income_stats.sort_values(by="Month", ascending=True, inplace=True)

print(monthly_income_stats)



    Month  Total_Transactions  Total_Income  Avg_Transaction_Amount
0       1                  26      14774.74              568.259231
1       2                  24       9637.62              401.567500
2       3                  28      16691.56              596.127143
3       4                  16       7266.69              454.168125
4       5                  15       7836.04              522.402667
5       6                  14       8043.69              574.549286
6       7                  22      10376.45              471.656818
7       8                  23      11833.55              514.502174
8       9                  19       8809.51              463.658421
9      10                  21      10731.39              511.018571
10     11                  25      11708.98              468.359200
11     12                  22       8559.83              389.083182


In [62]:
# find the total income and total expense for each region. Display the region, the total amount
# for the Income category, the total amount for the Expense category, and the difference between income and expense.
# Sort the results by the difference in descending order

grouped_data = df.groupby(["region", "category"]).agg(
    Total_Amount=("amount", "sum")
).reset_index()
pivoted_data = grouped_data.pivot(index="region", columns="category", values="Total_Amount").fillna(0)
pivoted_data["Difference"] = pivoted_data["Income"] - pivoted_data["Expense"]

result = pivoted_data.reset_index()

result_sorted = result.sort_values(by="Difference", ascending=False)

print(result_sorted)


category region   Expense    Income  Difference
2         South  31247.90  36611.68     5363.78
0          East  33071.90  35219.32     2147.42
1         North  28104.53  29845.30     1740.77
3          West  40080.65  24593.75   -15486.90


In [63]:
# find the top 3 months (based on the total transaction amount) along with the
# percentage contribution of each month’s total to the overall total transaction amount. Display the month name,
# total transaction amount, and percentage contribution

df["month_name"] = df["transaction_date"].dt.strftime("%B")

monthly_totals = df.groupby("month_name").agg(
    Total_Transaction_Amount=("amount", "sum")
).reset_index()

overall_total = monthly_totals["Total_Transaction_Amount"].sum()

monthly_totals["Percentage_Contribution"] = (
    monthly_totals["Total_Transaction_Amount"] / overall_total * 100
)

top_3_months = monthly_totals.sort_values(by="Total_Transaction_Amount", ascending=False).head(3)
print(top_3_months)



  month_name  Total_Transaction_Amount  Percentage_Contribution
7      March                  28402.07                10.975584
4    January                  26852.78                10.376882
9   November                  25360.51                 9.800215


In [50]:
df.head()

,transaction_id,customer_id,customer_name,transaction_date,amount,category,status,payment_method,region,merchant_name,month
0,1,116,Hannah Taylor,2024-08-27,61.37,Income,Completed,Credit Card,West,Merchant_14,August
1,2,120,George King,2024-05-08,546.56,Expense,Completed,Bank Transfer,West,Merchant_8,May
2,3,112,Jessica Lee,2024-01-18,908.56,Expense,Completed,PayPal,West,Merchant_39,January
3,4,113,George King,2024-05-30,976.36,Expense,Pending,PayPal,South,Merchant_7,May
4,5,105,Fiona Green,2024-12-03,261.62,Expense,Completed,Debit Card,East,Merchant_30,December
